In [1]:
""" %pip install pandas
%conda install pandas
%pip install unicodedata
%conda install unicodedata
%pip install os
%conda install os
%pip install openpyxl
%conda install openpyxl
%pip install numpy
%conda install numpy """

import pandas as pd
import unicodedata
import numpy as np
import os

In [ ]:


def leer_nombres_archivos():
    """Obtiene una lista de archivos en el directorio actual"""
    archivos = [archivo for archivo in os.listdir(os.getcwd())]
    return archivos

def tratar_texto(texto):
    """Normaliza cadenas eliminando tildes y caracteres especiales"""
    if isinstance(texto, str):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes
    elif isinstance(texto, pd.Timestamp):
        return texto.strftime("%d-%m-%Y")
    else:
        return texto  # Si no es texto, devuelve el valor original

def diccionario_archivos_excel():
    """Crea un diccionario con los datos de los archivos Excel procesados"""
    archivos_excel = {}
    for archivo in leer_nombres_archivos():
        # Solo procesar archivos con extensión .xlsx
        if archivo.endswith(".xlsx"):
            ruta_completa = os.path.join(os.getcwd(), archivo)
            try:
                # Cargar el archivo Excel
                df = pd.read_excel(
                    ruta_completa,
                    skiprows=7,
                    usecols="D:K",
                    engine="openpyxl"
                )
                # Renombrar columnas específicas
                df = df.rename(columns={
                    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
                    "FECHA POSESIÓN": "FECHA_POSESION",
                    "FECHA BAJA": "FECHA_BAJA",
                    "CÓDIGO INE": "CODIGO_NIE"
                })
                # Aplicar tratamiento de texto a cada celda
                df_sin_tildes = df.map(tratar_texto)
                # Guardar el DataFrame en el diccionario
                nombre_base = os.path.splitext(archivo)[0]
                archivos_excel[nombre_base] = df_sin_tildes
                print(f"Datos del archivo {archivo} cargados con éxito.")
            except Exception as e:
                print(f"Error al procesar el archivo {archivo}: {e}")
    return archivos_excel

# Procesar los archivos Excel
archivos_excel = diccionario_archivos_excel()

# Guardar los resultados en un único archivo Excel
archivo_salida = "prueba.xlsx"
with pd.ExcelWriter(archivo_salida, engine="openpyxl") as writer:
    for nombre_hoja, df in archivos_excel.items():
        # Guardar cada DataFrame en una hoja separada
        df.to_excel(writer, sheet_name=nombre_hoja, index=False)

print(f"Archivo combinado guardado como {archivo_salida}")


In [3]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
datos = pd.concat(hojas.values(), ignore_index=True)


1.En esta Burbuja vamos a saber que alcalde repetio el mandanto y en que lexislaturas estuvo.

In [ ]:
df_repetidos = df.groupby(["NOMBRE"]).size().reset_index(name="Veces_Elegido")
df_repetidos =  df_repetidos[df_repetidos["Veces_Elegido"] > 1].sort_values("Veces_Elegido")
print(df_repetidos)

2.En esta burbuja vamos a sacar los partidos con mas exito, para eso vamos a combinar todas las hojas en un solo dataFrame para obtener todos los datos de todas las hojas que tenemos.

In [ ]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
    
datos = pd.concat(hojas.values(), ignore_index=True)
df_partidos_exitosos = df.groupby(["LISTA"]).size().reset_index(name="Veces_Totales")
df_partidos_exitosos = df_partidos_exitosos.sort_values("Veces_Totales", ascending=False)
print(df_partidos_exitosos.head(5))

2.En esta burbuja vamos a comprobar los alcaldes que cambiaron de partido y tambien combinar todas las hojas en un solo DataFrame.

In [ ]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
datos = pd.concat(hojas.values(), ignore_index=True)

df_alcaldes_veletas = df.groupby("NOMBRE")["LISTA"].nunique().reset_index(name="Veces_Totales")
df_alcaldes_veletas = df_alcaldes_veletas[df_alcaldes_veletas["Veces_Totales"] > 1]
print(df_alcaldes_veletas)

3.En esta Burbuja vamos a calcular los alcades que cesaron antes de tiempo.

In [ ]:
ruta_archivo = "prueba.xlsx"
hojas = pd.read_excel(ruta_archivo, sheet_name=None)
datos = pd.concat(hojas.values(), ignore_index=True)

df_alcaldes_caidos = datos[datos["FECHA_BAJA"].notnull()] #datos.groupby("NOMBRE").size().reset_index()
df_alcaldes_caidos1 = df_alcaldes_caidos.groupby(["NOMBRE", "LISTA", "PROVINCIA", "COMUNIDAD_AUTONOMA"]).size()
df_alcaldes_caidos2 = df_alcaldes_caidos.groupby(["NOMBRE", "LISTA"]).size().reset_index(name="Veces_Totales")
df_alcaldes_caidos2 = df_alcaldes_caidos2[df_alcaldes_caidos2["Veces_Totales"] > 1]
print(df_alcaldes_caidos2)

4.En esta Burbuja vamos a calcular cual es el sueldo mas alto y el alcalde correspondiente.

In [ ]:
ruta_archivo_alcaldes = "Alcaldes_Mandato_2019_2023.xlsx"
ruta_archivo_retribuciones = "tabula-retribuciones_alcaldes_2021.csv"
df_alcaldes = pd.read_excel(
    ruta_archivo_alcaldes,
    skiprows=7,
    usecols="D:K",
    engine="openpyxl"
)
df_alcaldes = df_alcaldes.rename(columns={
    "COMUNIDAD AUTÓNOMA": "COMUNIDAD_AUTONOMA",
    "FECHA POSESIÓN": "FECHA_POSESION",
    "FECHA BAJA": "FECHA_BAJA",
    "CÓDIGO INE": "CODIGO_NIE"
})
df_alcaldes = df_alcaldes.map(tratar_texto)
df_retribuciones = pd.read_csv(ruta_archivo_retribuciones, skiprows=1)
df_retribuciones = df_retribuciones.rename(columns={"CCAA": "COMUNIDAD_AUTONOMA",
                                                    "DEDICACIÓN": "REGIMEN_DEDICACION",
                                                    "PERCIBIDO": "TOTAL_PERCIBIDO",
                                                    "AYUNTAMIENTO": "MUNICIPIO"
                                                    }
                                           )
df_retribuciones = df_retribuciones.map(tratar_texto)
df_retribuciones_alcaldes = pd.merge(df_alcaldes, df_retribuciones, on="MUNICIPIO", how="inner")

#Cómo vienen los números con separador de miles por "," pues hacemos los cambios necesarios
#convertimos a str, reemplazamos y dejamos el tipo a float
df_retribuciones_alcaldes["TOTAL_PERCIBIDO"] = df_retribuciones_alcaldes["TOTAL_PERCIBIDO"].astype(str).str.replace(".","").str.replace(",",".").astype(float)

#Se busca el índice de la tabla con el valor máximo para .loc[]alizar en el dataframe
df_retribuciones_alcaldes.loc[df_retribuciones_alcaldes["TOTAL_PERCIBIDO"].idxmax(),["NOMBRE","TOTAL_PERCIBIDO"]]


5.En esta Burbuja vamos a relacionar los sueldos altos y bajos con la probalidad de repetir mandato.